In [8]:
import nltk
from nltk.corpus import gutenberg

In [3]:
milton_paradise = gutenberg.raw('milton-paradise.txt')

In [29]:
milton_paradise[-10:]

u'he End]\x1a\x1a\n'

In [30]:
def text_preprocess(document):
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    return(sentences)

In [31]:
milton_paradise_sents = text_preprocess(milton_paradise)

In [33]:
import pandas as pd

In [52]:
pdf = pd.DataFrame({
        'sentences': nltk.sent_tokenize(milton_paradise)
    })
df = spark.createDataFrame(pdf)

In [55]:
df.show(n=5)

+--------------------+
|           sentences|
+--------------------+
|[Paradise Lost by...|
|And chiefly thou,...|
|Say first--for He...|
|Who first seduced...|
|Th' infernal Serp...|
+--------------------+
only showing top 5 rows



In [57]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [181]:
def sent_to_tag_words(sent):
    wordlist = nltk.word_tokenize(sent)
    tagged_words = nltk.pos_tag(wordlist)
    return(tagged_words)
schema = ArrayType(StructType([
            StructField('f1', StringType()),
            StructField('f2', StringType())
        ]))
sent_to_tag_words_udf = udf(sent_to_tag_words, schema)
# word_tokenize_udf = udf(lambda x: nltk.word_tokenize(x), ArrayType(StringType()))
# pos_tag_udf = udf(lambda x: nltk.pos_tag(x), ArrayType(ArrayType(StringType())))

In [ ]:
df_tagged_words = df.select(sent_to_tag_words_udf(df.sentences).alias('tagged_words'))
df_tagged_words.show(5)

In [168]:
# df_tokenized_words = df.select(word_tokenize_udf(df.sentences).alias('tokenized_words'))
# df_pos_tagged_words = df_tokenized_words.select(pos_tag_udf(df_tokenized_words.tokenized_words).alias('pos_tagged_words'))

In [175]:
# nltk.pos_tag(df_tokenized_words.head()['tokenized_words'])
# df_pos_tagged_words.show(truncate=False)

In [87]:
df.head()['sentences']

u"[Paradise Lost by John Milton 1667] \n \n \nBook I \n \n \nOf Man's first disobedience, and the fruit \nOf that forbidden tree whose mortal taste \nBrought death into the World, and all our woe, \nWith loss of Eden, till one greater Man \nRestore us, and regain the blissful seat, \nSing, Heavenly Muse, that, on the secret top \nOf Oreb, or of Sinai, didst inspire \nThat shepherd who first taught the chosen seed \nIn the beginning how the heavens and earth \nRose out of Chaos: or, if Sion hill \nDelight thee more, and Siloa's brook that flowed \nFast by the oracle of God, I thence \nInvoke thy aid to my adventurous song, \nThat with no middle flight intends to soar \nAbove th' Aonian mount, while it pursues \nThings unattempted yet in prose or rhyme."